In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
sys.path.append('../')
from bert_mpi import *

In [ ]:
from bert_mpi import *
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
filename = "mpi_small"
local_path = "../Dataset/" + f"{filename}.csv"
mpi = MPI(local_path)
mpi.run(tokenizer, model)